In [49]:
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
import os
import re
from dateutil.parser import parse

In [56]:
def load_driver(SELENIUM_EXECUTABLE_PATH=r'/mnt/c/Users/adity/Downloads/Chrome/geckodriver-v0.27.0-win64/geckodriver.exe'):
    driver = webdriver.Firefox(executable_path=SELENIUM_EXECUTABLE_PATH)
    return driver

def save_to_csv(data, SAVE_PATH, MODE):
    if not os.path.exists(SAVE_PATH.split('/')[0]):
        os.makedirs(SAVE_PATH.split('/')[0])

    fileWriter = csv.DictWriter(open(SAVE_PATH, MODE), data[0].keys(), delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    fileWriter.writeheader()
    fileWriter.writerows(data)

def validate_date(date):
    try:
        parse(date)
        return True
    except ValueError:
        return False

def format_registry_url(str_msg):
    try:
        url_ref = ''
        if 'href' in str_msg:
            href_subset = str_msg[str_msg.index('href')+6:]
            url_ref = '/'.join(BASE_PATH.split('/')[:-1]) + href_subset[:href_subset.index('">')]
        return url_ref
    except:
        return None

def get_particular_unique_rows(file_path, index):
    rows = []
    try:
        with open(file_path) as csvfile:
            data = csv.reader(csvfile, delimiter=',')
            for row in data:
                if row[index] not in rows:
                    rows.append(row[index])
    except:
        pass
    return rows

def persist_certification_details(certificate_id, SAVE_PATH='logs/certificate.csv'):
    BASE_PATH='https://www.psacard.com/cert'
    url = '/'.join([BASE_PATH, certificate_id])
    driver = load_driver()
    driver.get(url)
    soup=BeautifulSoup(driver.page_source)

    certificate_details = soup.find_all("div", attrs={"class": "cert-container"})[0].findAll('tr')[2:]
    if len(certificate_details) == 0:
        return
    certificate_map = {'certificate_number': certificate_id}
    ptr = 0
    if not len(certificate_details[0].contents[-1].contents[0]) == 4:
        certificate_map['reverse_cert_number'] = certificate_details[ptr].contents[-1].contents[0]
        ptr += 1
    else:
        certificate_map['reverse_cert_number'] = None

    certificate_map['year'] = certificate_details[ptr].contents[-1].contents[0]
    ptr += 1
    certificate_map['brand'] = certificate_details[ptr].contents[-1].contents[0]
    ptr += 1
    certificate_map['sport'] = certificate_details[ptr].contents[-1].contents[0]
    ptr += 1
    certificate_map['card_number'] = certificate_details[ptr].contents[-1].contents[0]
    ptr += 1
    certificate_map['player'] = certificate_details[ptr].contents[-1].contents[0]
    ptr += 1
    certificate_map['variety_or_pedigree'] = certificate_details[ptr].contents[-1].contents[0]
    ptr += 1
    certificate_map['grade'] = certificate_details[ptr].contents[-1].contents[0]
    ptr += 1

    # Adding PSA Auction Prices Realized
    realized_auction_prices = soup.find_all('table', attrs={"class": "apritem-results"})[0].findAll('tr')
    date = realized_auction_prices[0].contents[-1].contents[0]
    if validate_date(date):
        certificate_map['date'] = date
        certificate_map['price'] = realized_auction_prices[1].contents[-1].contents[0]
        certificate_map['auction_house'] = realized_auction_prices[2].contents[-1].contents[0]
        certificate_map['lot_number'] = realized_auction_prices[3].contents[-1].contents[0]
    else:
        certificate_map['date'] = certificate_map['price'] = certificate_map['auction_house'] = certificate_map['lot_number'] = None

    # Adding current PSA registry sets
    registry_sets = soup.find_all('div', attrs={"class": "col-xs-12"})
    certificate_map['registry_set_msg'] = str(soup.find_all('p')[3].contents)
    certificate_map['registry_set_url'] = format_registry_url(certificate_map['registry_set_msg'])
    certificate_map['population'] = registry_sets[0].find('span').contents[0]
    certificate_map['population_w_equal'] = registry_sets[1].find('span').contents[0]
    certificate_map['population_higher'] = registry_sets[2].find('span').contents[0]

    # Save to CSV
    save_to_csv([certificate_map], SAVE_PATH, 'a')
    driver.quit()
    return

In [51]:
TRANSACTION_FILE_PATH = 'logs/transaction.csv'
CERTIFICATION_FILE_PATH = 'logs/certificate.csv'
existing_certificates_in_transaction = set(get_particular_unique_rows(TRANSACTION_FILE_PATH, 11))
existing_certificates_in_certificates = set(get_particular_unique_rows(CERTIFICATION_FILE_PATH, 0))
new_certificates = existing_certificates_in_transaction - existing_certificates_in_certificates

In [52]:
# for new_certificate in new_certificates:
#     persist_certification_details(new_certificate)

In [53]:
certificate_id = list(new_certificates)[0]
BASE_PATH='https://www.psacard.com/cert'
url = '/'.join([BASE_PATH, certificate_id])
driver = load_driver()certificate_details
driver.get(url)
soup=BeautifulSoup(driver.page_source)

certificate_details = soup.find_all("div", attrs={"class": "cert-container"})[0].findAll('tr')[2:]
certificate_map = {'certificate_number': certificate_id}

In [57]:
len(certificate_details) == 0

True

In [42]:
ptr = 0
if not len(certificate_details[0].contents[-1].contents[0]) == 4:
    certificate_map['reverse_cert_number'] = certificate_details[ptr].contents[-1].contents[0]
    ptr += 1
else:
    certificate_map['reverse_cert_number'] = None

certificate_map['year'] = certificate_details[ptr].contents[-1].contents[0]
ptr += 1
certificate_map['brand'] = certificate_details[ptr].contents[-1].contents[0]
ptr += 1
certificate_map['sport'] = certificate_details[ptr].contents[-1].contents[0]
ptr += 1
certificate_map['card_number'] = certificate_details[ptr].contents[-1].contents[0]
ptr += 1
certificate_map['player'] = certificate_details[ptr].contents[-1].contents[0]
ptr += 1
certificate_map['variety_or_pedigree'] = certificate_details[ptr].contents[-1].contents[0]
ptr += 1
certificate_map['grade'] = certificate_details[ptr].contents[-1].contents[0]
ptr += 1

In [54]:
certificate_map

{'certificate_number': 'cert'}

In [39]:
len(certificate_details[0].contents[-1].contents[0])

4

In [6]:
for new_certificate in new_certificates:
    persist_certification_details(new_certificate)

#print ("Total New Certificates Added = ", len(new_certificates))

AttributeError: 'NavigableString' object has no attribute 'contents'

In [9]:
existing_certificates_in_certificates

set()

In [ ]:
def main():
    TRANSACTION_FILE_PATH = 'logs/transaction.csv'
    CERTIFICATION_FILE_PATH = 'logs/certificate.csv'
    existing_certificates_in_transaction = set(get_particular_unique_rows(TRANSACTION_FILE_PATH, 11))
    existing_certificates_in_certificates = set(get_particular_unique_rows(CERTIFICATION_FILE_PATH, 0))
    new_certificates = existing_certificates_in_transaction - existing_certificates_in_certificates
    for new_certificate in new_certificates:
        persist_certification_details(new_certificate)

    print ("Total New Certificates Added = ", len(new_certificates))

if __name__ == '__main__':
    main()


In [ ]:
def load_driver(SELENIUM_EXECUTABLE_PATH=r'/mnt/c/Users/adity/Downloads/Chrome/geckodriver-v0.27.0-win64/geckodriver.exe'):
    driver = webdriver.Firefox(executable_path=SELENIUM_EXECUTABLE_PATH)
    return driver

def save_to_csv(data, SAVE_PATH, MODE):
    flag = os.path.isfile(SAVE_PATH)
    with open(SAVE_PATH, MODE) as file:
        fileWriter = csv.DictWriter(file, fieldnames=data[0].keys(), delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        if not flag:
            fileWriter.writeheader()
        fileWriter.writerows(data)

def parse_amount_details(rel_url, global_identifiers, tag='None', SAVE_PATH='logs/transaction.csv', MODE='a'):
    BASE_PATH='https://www.psacard.com'
    identifiers = rel_url['href'].split('/')[3:]
    auction_name = identifiers[0]
    auction_player = identifiers[1]
    value = rel_url.text
    summary_id = re.sub(r'#.*', '', identifiers[-1])

    url = BASE_PATH + most_recent_prices['href']
    driver = load_driver()
    driver.get(url)
    soup=BeautifulSoup(driver.page_source, features="lxml")
    all_details = soup.find_all("div", attrs={"class": "item-lots-summary"})[0].find_all("div", attrs={"class": "item-row"})
    amount_arr = []
    for fine_details in all_details[1:]:
        specifics = {}
        specifics['auction_name'] = auction_name
        specifics['auction_player'] = auction_player
        specifics['value'] = value
        specifics['summary_id'] = summary_id
        specifics['date'] = fine_details['data-date']
        specifics['price'] = fine_details['data-price']
        specifics['grade'] = fine_details['data-gradevalue']
        specifics['lot_number'] = fine_details['data-lot']
        specifics['auction_house'] = fine_details['data-auctionhouse']
        specifics['auction_seller'] = fine_details['data-auctionname']
        specifics['auction_type'] = fine_details['data-auctiontype']
        specifics['cert'] = fine_details['data-cert']
        specifics['tag'] = tag

        specifics['fk_name'], specifics['fk_url'], specifics['fk_count'], specifics['fk_category'] = global_identifiers
        amount_arr.append(specifics)

    # Save to File
    return amount_arr
    print (">", amount_arr)
    save_to_csv(amount_arr, SAVE_PATH, MODE)
    driver.quit()
    return

In [ ]:
def parse_population_details(population, global_identifiers, tag=None, SAVE_PATH='logs/population_report.csv', MODE='a'):
    BASE_PATH='https://www.psacard.com'
    rel_url = population['href'].split('/')[3:]
    auction_year = rel_url[0]
    population_id = rel_url[-1]
    url = BASE_PATH + population['href']
    driver = load_driver()
    driver.get(url)
    soup=BeautifulSoup(driver.page_source, features="lxml")
    driver.quit()
    all_details = soup.find_all("div", attrs={"class": "dataTables_wrapper"})[0]

    population_arr = []
    for row in all_details.findAll('tr')[1:]:
        population_row = {}
        all_td = row.findAll('td')[1:]
        metrices = [th.text.strip() for th in all_details.findAll('th')[3:]]
        auction_meta = str(all_td[0])
        population_row['auction_player'] = auction_meta[auction_meta.index('<strong>')+len('<strong>'):auction_meta.index('</strong>')].strip()
        population_row['auction_league'] = auction_meta[auction_meta.index('<br/>')+8: auction_meta.index('<span')-3].strip()
        population_row['population_id'] = population_id
        population_row['auction_year'] = auction_year
        population_row['fk_name'], population_row['fk_url'], population_row['fk_count'], population_row['fk_category'] = global_identifiers

        # Collecting all metrices for the player
        idx = 0
        for item in all_td[2:]:
            cells = item.findAll('span')
            population_row[metrices[idx]] = {'Grade': cells[0].contents[0], '+': cells[1].contents[0], 'Q': cells[2].contents[0]}
            idx += 1
        population_arr.append(population_row)
        

    # Append to the csv file
    save_to_csv(population_arr, SAVE_PATH, MODE)
    return

In [ ]:
CSV_FILE_PATH='logs/allauctionprices.csv'
with open(CSV_FILE_PATH) as csvfile:
    auction_items= csv.reader(csvfile, delimiter=',')
    cnt = 0
    for row in auction_items:
        name, url, count, category = row
        url = 'https://www.psacard.com/auctionprices/basketball-cards/1994-skybox-usa-basketball/shawn-kemp/summary/2684176'
        global_identifiers = [name, url, count, category]
        driver = load_driver()
        driver.get(url)
        soup=BeautifulSoup(driver.page_source, features="lxml")
        grade_prices = soup.find_all("table", attrs={"class": "set-items-results"})
        for idx in range(len(grade_prices)-1):
            specifics = grade_prices[idx].findAll('tr')

            grade = specifics[0].findAll('td')[-1].contents[0]
            most_recent_prices = specifics[1].findAll('td')[-1].contents[0]
            average_price = specifics[2].findAll('td')[-1].contents[0]
            smr_price = specifics[3].findAll('td')[-1].contents[0]
            population = specifics[4].findAll('td')[-1].contents[0]
            pop_higher = specifics[5].findAll('td')[-1].contents[0]

            if str(population) != '—':
                parse_population_details(population, global_identifiers, 'Population')

In [ ]:

for idx in range(len(grade_prices)-1):
    specifics = grade_prices[idx].findAll('tr')

    grade = specifics[0].findAll('td')[-1].contents[0]
    most_recent_prices = specifics[1].findAll('td')[-1].contents[0]
    average_price = specifics[2].findAll('td')[-1].contents[0]
    smr_price = specifics[3].findAll('td')[-1].contents[0]
    population = specifics[4].findAll('td')[-1].contents[0]
    pop_higher = specifics[5].findAll('td')[-1].contents[0]
    
    print(idx, most_recent_prices, average_price, smr_price)
    
#     if str(most_recent_prices) != '—':
#         parse_amount_details(most_recent_prices, global_identifiers, 'Most Recent Price')

#     if str(average_price) != '—':
#         parse_amount_details(average_price, global_identifiers, 'Average Price')

#     if str(smr_price) != '—':
#         parse_amount_details(most_recent_prices, global_identifiers, 'SMR Price' )

In [ ]:
specifics = grade_prices[3].findAll('tr')
grade = specifics[0].findAll('td')[-1].contents[0]
most_recent_prices = specifics[1].findAll('td')[-1].contents[0]
average_price = specifics[2].findAll('td')[-1].contents[0]
smr_price = specifics[3].findAll('td')[-1].contents[0]
population = specifics[4].findAll('td')[-1].contents[0]
pop_higher = specifics[5].findAll('td')[-1].contents[0]

In [ ]:
most_recent_prices

In [ ]:
amount_arr = parse_amount_details(most_recent_prices, global_identifiers, 'Most Recent Price')

In [ ]:
global_identifiers

In [ ]:
grade = specifics[0].findAll('td')[-1].contents[0]

In [ ]:
grade_prices[12].findAll('tr')[1:].findAll('td')[-1]

In [ ]:
name, url, count, category


In [ ]:
CSV_FILE_PATH='logs/allauctionprices.csv'
with open(CSV_FILE_PATH) as csvfile:
    auction_items= csv.reader(csvfile, delimiter=',')
    for row in auction_items:
        try:
            name, url, count, category = row
            global_identifiers = [name, url, count, category]
            driver = load_driver()
            driver.get(url)
            soup=BeautifulSoup(driver.page_source, features="lxml")
            grade_prices = soup.find_all("table", attrs={"class": "set-items-results"})
            unit = grade_prices[4]
            specifics = unit.findAll('tr')
            grade = specifics[0].findAll('td')[-1].contents[0]
            most_recent_prices = specifics[1].findAll('td')[-1].contents[0]
            average_price = specifics[2].findAll('td')[-1].contents[0]
            smr_price = specifics[3].findAll('td')[-1].contents[0]
            population = specifics[4].findAll('td')[-1].contents[0]
            pop_higher = specifics[5].findAll('td')[-1].contents[0]

            if str(most_recent_prices) != '—':
                parse_amount_details(most_recent_prices, global_identifiers, 'Most Recent Price')

            if str(average_price) != '—':
                parse_amount_details(average_price, global_identifiers, 'Average Price')

            if str(smr_price) != '—':
                parse_amount_details(most_recent_prices, global_identifiers, 'SMR Price' )

            driver.quit()
        except:
            print ("Error ....")


In [ ]:
def main():
    
if __name__ == '__main__':
    main()
